<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 6. Custom Interactivity</h2></div>

<div class="alert alert-warning" role="alert"> <strong>WORK IN PROGRESS:</strong> We are in the progress of updating these materials in anticipation of a tutorial at the 2019 SciPy conference. Work will be complete by the morning of July 8th 2019. Check out <a href="https://github.com/pyviz/holoviz/tree/v0.1.1">this tag</a> to access the materials as they were before these changes started. For the latest version of the tutorial, visit <a href="https://holoviz.org/tutorial">holoviz.org</a>.
</div>

Using hvplot allows you to generate a number of different types of plot
quickly by building [HoloViews](https://holoviews.org) objects as discussed in the previous
notebook. These objects are rendered with Bokeh which offers a number of
standard ways to interact with your plot, such as panning and zooming
tools.

Many other modes of interactivity are possible when building an
exploratory visualization (such as a dashboard) and these forms of
interactivity cannot be achieved using hvplot alone.

In this notebook, we will drop down to the HoloViews level of
representation to build a visualization consisting of linked plots that
update when you interactivity select a particular earthquake with the
mouse. The goal is to show how more sophisticated forms or interactivity
can be introduced in the spirit of "shortcuts" not "dead ends".

First let us load our initial imports:

In [ ]:
import numpy as np
import dask.dataframe as dd
import hvplot.pandas  # noqa


And clean the data as before:

In [ ]:
df = dd.read_parquet('../data/earthquakes.parq')
cleaned_df = df.copy()
cleaned_df['mag'] = df.mag.where(df.mag > 0)
cleaned_reindexed_df = cleaned_df.set_index(cleaned_df.time)
cleaned_reindexed_df = cleaned_reindexed_df.persist()


In the previous notebook we generated a scatter plot of earthquakes
across the earth that had a magnitude `>7`:

In [ ]:
most_severe = df[df.mag >= 7].compute()
high_mag_scatter = most_severe.hvplot.scatter(x='longitude', y='latitude', c='mag')
high_mag_scatter


And saw how this object is a HoloViews `Scatter` object:

In [ ]:
print(high_mag_scatter)


This object is an example of a HoloViews *Element*

### HoloViews Elements

HoloViews elements are the atomic, visualizable components that can be
rendered by a plotting library such as Bokeh. We don't need to use
hvplot to create these element objects: we can create them directly by
importing HoloViews (and loading the extension if we have not loaded
hvplot):

In [ ]:
import holoviews as hv
hv.extension("bokeh") # Optional here as we have already loaded hvplot.pandas


Now we can create our own example of a `Scatter` element. In the next
cell we plot 100 points with a normal (independent) distrbutions in the
`x` and `y` directions:

In [ ]:
xs = np.random.randn(100)
ys = np.random.randn(100)
hv.Scatter((xs, ys))


Now that the axis labels are 'x' and 'y', the default *dimensions* for
this element type. We can use a different set of dimensions, say
'weight' and 'height' if we wish:

In [ ]:
xs = np.random.randn(100)
ys = np.random.randn(100)
height_v_weight = hv.Scatter((xs, ys), 'weight', 'height')
height_v_weight

For more information an HoloViews dimensions, see this [user guide](http://holoviews.org/user_guide/Annotating_Data.html).

In [ ]:
### Exercise: Visit http://holoviews.org/reference/index.html and browse
### the available set of elements. Pick an element type and try running
### one of the self-contained examples in this cell.


### Help and Options

The two `Scatter` elements above look quite different from the one
returned by hvplot showing the earthquake positions. This is because
hvplot makes use of the HoloViews *options system* to customize the
visual representation of these element objects.

Let us make the `height_v_weight` scatter above green with larger
points:

In [ ]:
height_v_weight.opts(color='green', size=8)


You can learn more about the `.opts` method and the HoloViews options
system in the [corresponding user
guide](http://holoviews.org/user_guide/Applying_Customizations.html). To
easily learn about the available options from inside a notebook, you can
use `hv.help` and inspect the 'Style Options'.

In [ ]:
### Exercise: Uncomment the line below and run this cell. Try inspecting
### the output for different element types.
# hv.help(hv.Scatter)



At this point, we can have some insight to the sort of HoloViews object
hvplot is building behind the scenes for our earthquake example:

In [ ]:
hv.Scatter(most_severe, 'longitude', ['latitude', 'mag']).opts(color='mag', aspect='equal')


### Adding interactivity to Elements

When rasterization of the population density data via hvplot was
introduced in the last notebook, we saw that the HoloViews object
returned was not an element but a *`DynamicMap`*.

A `DynamicMap` enables custom interactivity beyond the Bokeh defaults by
dynamically generating elements that get displayed and updated as the
plot is interacted with.

There is a counterpart to the `DynamicMap` that does not require a live
Python server to be running called the `HoloMap`. The `HoloMap`
container will not be covered in the tutorial but you can learn more
about them in the [containers user
guide](http://holoviews.org/user_guide/Dimensioned_Containers.html).

Now let us build a very simple `DynamicMap` that is driven by a *linked
stream* specifically a `PointerXY` stream that represents the position
of the cursor over the plot:

In [ ]:
from holoviews import streams
pointer = streams.PointerXY(x=0, y=0) # x=0 and y=0 are the initialized values

def crosshair(x, y):
    return  hv.Ellipse(0,0,1) * hv.HLine(y) * hv.VLine(x)

hv.DynamicMap(crosshair, streams=[pointer])

Try moving your mouse over the plot and you should see the crosshair
follow your mouse position.

The core concepts here are:

* The plot shows an overlay built with the `*` operator introduced in
  the previous notebook.
* There is a callback that returns this overlay that is built according
  to the supplied `x` and `y` arguments. A DynamicMap always contains a
  callback that returns a HoloViews object such as an `Element` or
  `Overlay`
* These `x` and `y` arguments are supplied by the `PointerXY` stream
  that reflect the position of the mouse on the plot.

In [ ]:
### Exercise: Look up the Ellipse, HLine and VLine elements in the
### reference guide (http://holoviews.org/reference/index.html) and see
### if they align with your intuitions.

### Additional Exercise: If you have time, try running one of the examples in the
### 'Streams' section of the HoloViews reference guide
### (http://holoviews.org/reference/index.html) in this cell

### Selecting a particular earthquake with the mouse

Now we only need two more concept before we can set up the appropriate
mechanism to select a particular earthquake on the hvplot generated
Scatter plot we started with.

First, we can attach a stream to an existing HoloViews element such as
the earthquake distribution generated with hvplot:

In [ ]:
selection_stream = streams.Selection1D(source=high_mag_scatter)

Next we need to enable the 'tap' tool on our Scatter to instruct Bokeh
to enable the desired selection mechanism in the browser.

In [ ]:
high_mag_scatter.opts(tools=['tap'])

The tap tool is in the toolbar with the icon showing the concentric
circles and plus symbol. If you enable this tool, you should be able to pick individual earthquakes above by tapping on them.

Now we can make a DynamicMap that uses the stream we defined to show the index of the earthquake selected via the `hv.Text` element:

In [ ]:
def labelled_callback(index):
    if len(index) == 0:
        return  hv.Text(x=0,y=0, text='')
    first_index = index[0] # Pick only the first one if multiple are selected
    row = most_severe.iloc[first_index]
    return hv.Text(x=row.longitude,y=row.latitude,text='%d : %s' % (first_index, row.place))

labeller = hv.DynamicMap(labelled_callback, streams=[selection_stream])

This labeller receives the index argument from the Selection1D stream
which corresponds to the row of the original dataframe (`most_severe`)
that was selected. This lets us present the index and place value using
`hv.Text` which we then position at the corresponding latitude and
longitude to label the chosen earthquake.

Finally, we overlay this labeller `DynamicMap` over the original
plot. Now by using the tap tool you can see the index number of an
earthquake followed by the assigned place name:

In [ ]:
(high_mag_scatter.opts(tools=['tap']) * labeller).opts(hv.opts.Scatter(tools=['hover']))

In [ ]:
### Exercise: Pick an earthquake point above and using the displayed
### index, display the corresponding row of the `most_severe` dataframe
### using the `.iloc` method.

### Building a linked earthquake visualizer

Now we will build a visualization that achieves the following:

* The user shall select an earthquake with magnitude `>7` using the tap
  tool in the manner illustrated in the last section.

* *All* earthquakes within 0.5 degrees of latitude and longitude of the
   selected earthquake will be plotted in grey. 0.5 degrees across the
   Earth's surface is roughly 50km.

* Then we will generate histogram showing the distribution of magnitudes
  in the selected area.

* In addition, we will generate a timeseries scatter plot showing the magnitudes of earthquakes over time in the selected area.

The first step is to plot the lower magnitude quakes in the selected
area using an approach that is very similar to what is already shown
above. First we will define a function that given a latitude and
longitude, returns the rows of `most_severe` corresponding to earthquakes within 0.5 degrees of that point:

In [ ]:
def earthquake_around_point(df, lat, lon, degrees_dist=0.5):
    half_dist = degrees_dist / 2.0
    lat_mask = np.abs(df['latitude'] - lat) < half_dist
    lon_mask = np.abs(df['longitude'] - lon) < half_dist
    mask = np.bitwise_and(lat_mask.compute(), lon_mask.compute())
    return df[mask].compute()

We can now use this function to create a `DynamicMap` that reveals the
lower magnitude tremors around a particular earthquake:

In [ ]:
def localized_callback(index):
    if len(index) == 0:
        return  hv.Scatter(None, 'longitude', 'latitude')
    first_index = index[0] # Pick only the first one if multiple are selected
    row = most_severe.iloc[first_index]
    selected_df = earthquake_around_point(cleaned_reindexed_df, row.latitude, row.longitude)
    return hv.Scatter(selected_df, 'longitude', 'latitude').opts(color='green')

localized_earthquakes = hv.DynamicMap(localized_callback, streams=[selection_stream])

In [ ]:
high_mag_scatter.opts(tools=['tap']) * localized_earthquakes


Note that you may need to zoom in to your selected earthquake to see the
localized, lower magnitude earthquakes around it.

### Linked plots

So far we have overlayed the display updates on top of the existing
spatial distribution of earthquakes. However, there is no requirement
that the data is overlaid and we might want to simply attach an entirely
new, derived plot that dynamically updates to the side.

Using the same principles as we have already seen, we can define a
`DynamicMap` that returns `Histogram` distributions of earthquake
magnitude:

In [ ]:
def histogram_callback(index):
    if len(index) == 0:
        return  hv.Histogram(([], []), kdims=['longitude'], vdims=['latitude'])
    first_index = index[0] # Pick only the first one if multiple are selected
    row = most_severe.iloc[first_index]
    selected_df = earthquake_around_point(cleaned_reindexed_df, row.latitude, row.longitude)
    return selected_df.hvplot.hist(y='mag', bin_range=(0,10), bins=50)


histogram = hv.DynamicMap(histogram_callback, streams=[selection_stream])

The only real difference in the approach here is that we can still use
`.hvplot` to generate our elements instead of declaring the HoloViews
elements explicitly. In this example, `.hvplot.hist` is used.

Now we can combine the components we have already built as follows to create a dynamically updating plot together with an associated, linked histogram:

In [ ]:

((high_mag_scatter.opts(tools=['tap']) * labeller) + histogram).cols(1)



## TODO

This tutorial notebook is still in the process of being updated. The
following two sections will be added shortly:

* A second example of a linked plot showing a timeseries based on the selected set of earthquakes.

* A conclusion and summary.